<h1 style="text-align: center;">Práctica 4: BLAST </h1>
</h3>
<h3 style="text-align: center;"> Gisela Belmonte Cruz; Diego Marrero Ferrera </h3>


En esta práctica trabajaremos con BLAST (Basic Local Alignment Search Tool), una herramienta fundamental en bioinformática que permite comparar una secuencia biológica con millones de secuencias almacenadas en bases de datos internacionales.

Nuestro objetivo es aprender a usar BLAST online y BLAST local, además de procesar sus resultados utilizando Biopython. Para los recursos utilizados se han usado los archivos y conexiones que ofrece [NCBI](https://blast.ncbi.nlm.nih.gov/Blast.cgi).

Cada variante del algoritmo permite comparar secuencias de ADN, ARN o proteínas para identificar genes homólogos, funciones posibles o relaciones evolutivas. La práctica nos introduce a:

- Ejecutar BLAST desde Python,
- Procesar salidas en formato XML,
- Analizar resultados (E-value, identidad, etc.),
- Y trabajar con bases de datos locales o remotas.

---

## Ejercicio 1: BLAST a partir de una secuencia de ADN introducida por teclado.

En el Ejercicio 1 se nos pide realizar una búsqueda BLAST de ADN, introducida por teclado o leída desde un archivo FASTA, mostrando tres resultados:
1. Número total de alineamientos obtenidos
2. El E-value del mejor resultado
3. La descripción de la secuencia más similar

Trabajaremos tanto en versión online como local.

Para este ejercicio utilizaremos el archivo `rna.fna`, que contiene la secuencia de **ARNm del gen de la oxitocina humana**. Elegimos este archivo porque:
- Representa la versión ya procesada del gen,
- Es más corto y limpio que la secuencia genómica (gene.fna),
- Y es ideal para realizar una consulta `blastn`.

A continuación, cargaremos esa secuencia y ejecutaremos BLAST de manera online utilizando los servidores del NCBI.

In [6]:
from Bio import SeqIO

def leer_secuencias_rna(archivo):
    # Leer todas las secuencias del archivo
    registros = list(SeqIO.parse(archivo, "fasta"))
    print("Número de secuencias encontradas:", len(registros))

    # Mostramos la info de cada una
    for i, r in enumerate(registros):
        print(f"{i}: {r.id} (longitud: {len(r.seq)})")

    # Elegimos la primera secuencia porque es la más completa
    registro = registros[0]
    secuencia_query = str(registro.seq)

    print("\nSecuencia seleccionada:")
    print("ID:", registro.id)
    print("Longitud:", len(secuencia_query))
    print("Secuencia:", secuencia_query)
    return secuencia_query

secuencia_query = leer_secuencias_rna("./local/rna_oxi.fna")

Número de secuencias encontradas: 6
0: NM_000916.4 (longitud: 4387)
1: NM_001354653.2 (longitud: 4307)
2: NM_001354654.2 (longitud: 4364)
3: NM_001354655.2 (longitud: 4189)
4: NM_001354656.3 (longitud: 4048)
5: XR_007095681.1 (longitud: 3218)

Secuencia seleccionada:
ID: NM_000916.4
Longitud: 4387
Secuencia: GTTAAGGCTCTGGGACCAACGCTGGGCGAACCAGCTCCGCTCCGGAGGGGTCTGCGCGGCTGGCCTCGCCCGCCCCCTAGCGGACCCGTGCGATAGTGCAGCCTCAGCCCCAGCGCACAGCGCCGCATCCAGACGCTGTCCGCGCGCGCAGCCTGGGAGGCGCTCCTCGCTCGCCTCCTGTACCCATCCAGCGACCAGCCAGGCTGCGGCGAGGGGATTCCAACCGAGGCTCCAGTGAGAGACCTCAGCTTAGCATCACATTAGGTGCAGCCGGCAGGCCATCCCAACTCGGGCCGGGAGCGCACGCGTCACTGGGGCCGTCAGTCGCCGTGCAACTTCCCCGGGGGGAGTCAACTTTAGGTTCGCCTGCGGACTCGGTGCAGTGGAAGCCGCTGAACATCCCGAGGAACTGGCACGCTGGGGGCTCTGGGCTTGTGGCCGGTAGAGGATTCCCGCTCATTTGCAGTGGCTCAGAGGAGGGTGGACCCAGCAGATCCGTCCGTGGAGTCTCCAGGAGTGGAGCCCCGGGCGCCCCTACACCCTCCGACACGCCGGATCCGGCCCAGCCGCGCCAAGCCGTAAAGGGCTCGAAGGCCGGGGCGCACCGCTGCCGCCAGGGTCATGGAGGGCGCGCTCGCAGCCAACTGGAGCGCCGAGGCAGCCAACGCCAGCGCCGCGCCGCCGGGGGCC

Sabiendo la secuencia a buscar, ejecutamos BLAST online en NCBI:

In [ ]:
from Bio.Blast import NCBIWWW

print("Realizando búsqueda BLAST (blastn)...")

# Descomentar esto para ejecutar BLAST en línea (requiere conexión a internet y bastante tiempo de ejecución)
# ---
# resultado_xml = NCBIWWW.qblast(
#     program="blastn",
#     database="nt",
#     sequence=secuencia_query
# )

# # Guardar en archivo, en la carpeta "xml_creados"
# with open("./xml_creados/resultado_ej1_oxitocina.xml", "w") as f:
#     f.write(resultado_xml.read())
# ---

print("BLAST completado. Archivo guardado como resultado_ej1_oxitocina.xml")


Realizando búsqueda BLAST (blastn)...
BLAST completado. Archivo guardado como resultado_ej1_oxitocina.xml


Con el archivo XML que obtuvimos, lo procesamos y podemos mostrar los resultados solicitados:

In [ ]:
from Bio.Blast import NCBIXML

def analizar_resultados_blast(archivo_xml):
    with open(archivo_xml) as f:
        blast_records = list(NCBIXML.parse(f))

    record = blast_records[0]

    print("Número total de alineamientos con E-value < 1e-10 (Alineamientos casi o totalmente perfectos):", sum(1 for a in record.alignments if a.hsps[0].expect < 1e-10))
    print("-------------------------------")
    # Mejor alineamiento:
    best = record.alignments[0]
    best_hsp = best.hsps[0]

    print("=== MEJOR RESULTADO ===")
    print("Descripción:", best.hit_def)
    print("E-value:", best_hsp.expect)
    print("Longitud del alineamiento:", best.length)
    print("-------------------------------")
    print("Podio de mejores alineamientos:")
    for i, alignment in enumerate(record.alignments[:3]):
        print(f"  {i + 1}. {alignment.hit_def} (E-value: {alignment.hsps[0].expect})")
    print("-------------------------------")
    return record

record = analizar_resultados_blast("resultado_ej1_oxitocina.xml")


Número total de alineamientos con E-value < 1e-10 (Alineamientos casi o totalmente perfectos): 50
-------------------------------
=== MEJOR RESULTADO ===
Descripción: Homo sapiens oxytocin receptor (OXTR), transcript variant 1, mRNA
E-value: 0.0
Longitud del alineamiento: 4387
-------------------------------
Podio de mejores alineamientos:
  1. Homo sapiens oxytocin receptor (OXTR), transcript variant 1, mRNA (E-value: 0.0)
  2. Homo sapiens oxytocin receptor (OXTR), transcript variant 3, mRNA (E-value: 0.0)
  3. PREDICTED: Pan troglodytes oxytocin receptor (OXTR), transcript variant X1, mRNA (E-value: 0.0)
-------------------------------
